In [35]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory


from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [36]:
mx.context.num_gpus()

0

In [37]:
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
# df = pd.read_csv('pollution.csv')

In [38]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [39]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [40]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [41]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [42]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [43]:
train_df.shape

(77145, 21)

In [44]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [45]:
train_ds = PandasDataset(
    train_df,
    target = ['TWS_S26'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S25B', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

In [46]:
test_ds = PandasDataset(
    test_df,
    target = ['TWS_S26'],
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S25B', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [47]:
K = 48
n_hours = 24*3

In [48]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
)

model_26 = estimator.train(train_ds)

100%|███████████████████████████████████████████████| 100/100 [05:28<00:00,  3.29s/it, epoch=132/150, avg_epoch_loss=0.0316]


100%|███████████████████████████████████████████████| 100/100 [07:41<00:00,  4.61s/it, epoch=150/150, avg_epoch_loss=0.0321]


#### predicted y

In [49]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [50]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S26',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S1', 'TWS_S25A', 'TWS_S25B', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model_26, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0.74313277 0.25665909
 0.05603945 0.22033411 0.79564714 1.44398463 1.98395431 2.48369431]
(48,)
------
1
[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0

[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0.74313277 0.25665909
 0.05603945 0.22033411 0.79564714 1.44398463 1.98395431 2.48369431
 2.0324924  1.50960457 1.02110279 0.53950405 0.32399324 0.4993751
 1.03678083 1.63141668 2.12082982 2.54747581 2.71403599 2.55012417
 2.040766   1.47410297 0.96508646 0.46053386 0.14106727 0.17570503
 0.6460464  1.30219412 1.86458623 2.39635301 2.76107407 2.8113625
 2.45195532 1.83597672 1.32626975 0.83210534 0.41267362 0.27407745
 0.55704463 1.164976   1.72903872 2.23030949 2.59118581 2.6693294
 2.37822032 1.79680669 1.29262698 0.76683664 0.27276459 0.06426819

[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0.74313277 0.25665909
 0.05603945 0.22033411 0.79564714 1.44398463 1.98395431 2.48369431
 2.0324924  1.50960457 1.02110279 0.53950405 0.32399324 0.4993751
 1.03678083 1.63141668 2.12082982 2.54747581 2.71403599 2.55012417
 2.040766   1.47410297 0.96508646 0.46053386 0.14106727 0.17570503
 0.6460464  1.30219412 1.86458623 2.39635301 2.76107407 2.8113625
 2.45195532 1.83597672 1.32626975 0.83210534 0.41267362 0.27407745
 0.55704463 1.164976   1.72903872 2.23030949 2.59118581 2.6693294
 2.37822032 1.79680669 1.29262698 0.76683664 0.27276459 0.06426819

[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0.74313277 0.25665909
 0.05603945 0.22033411 0.79564714 1.44398463 1.98395431 2.48369431
 2.0324924  1.50960457 1.02110279 0.53950405 0.32399324 0.4993751
 1.03678083 1.63141668 2.12082982 2.54747581 2.71403599 2.55012417
 2.040766   1.47410297 0.96508646 0.46053386 0.14106727 0.17570503
 0.6460464  1.30219412 1.86458623 2.39635301 2.76107407 2.8113625
 2.45195532 1.83597672 1.32626975 0.83210534 0.41267362 0.27407745
 0.55704463 1.164976   1.72903872 2.23030949 2.59118581 2.6693294
 2.37822032 1.79680669 1.29262698 0.76683664 0.27276459 0.06426819

[2.51323962 1.98924065 1.47266853 0.98042303 0.51623917 0.31311059
 0.4915283  1.02235365 1.60488594 2.09089375 2.51266265 2.67695332
 2.51126575 1.99715519 1.43865931 0.93535393 0.43923306 0.13391285
 0.17786956 0.6562708  1.31081212 1.86701143 2.39162993 2.7447567
 2.78552365 2.42154932 1.80927503 1.30479717 0.81478333 0.39877805
 0.26570162 0.55420023 1.15904009 1.71582699 2.21274495 2.56378078
 2.63389587 2.33708477 1.76076174 1.26424015 0.74313277 0.25665909
 0.05603945 0.22033411 0.79564714 1.44398463 1.98395431 2.48369431
 2.0324924  1.50960457 1.02110279 0.53950405 0.32399324 0.4993751
 1.03678083 1.63141668 2.12082982 2.54747581 2.71403599 2.55012417
 2.040766   1.47410297 0.96508646 0.46053386 0.14106727 0.17570503
 0.6460464  1.30219412 1.86458623 2.39635301 2.76107407 2.8113625
 2.45195532 1.83597672 1.32626975 0.83210534 0.41267362 0.27407745
 0.55704463 1.164976   1.72903872 2.23030949 2.59118581 2.6693294
 2.37822032 1.79680669 1.29262698 0.76683664 0.27276459 0.06426819

...

[2.51323962 1.98924065 1.47266853 ... 2.5055058  2.65419579 2.47772408]
(262464,)
------
5468
[2.51323962 1.98924065 1.47266853 ... 2.66373658 2.47741008 2.00395083]
(262512,)
------
5469
[2.51323962 1.98924065 1.47266853 ... 2.4410603  1.97311699 1.50415647]
(262560,)
------
5470
[2.51323962 1.98924065 1.47266853 ... 1.93284929 1.48168278 1.0471251 ]
(262608,)
------
5471
[2.51323962 1.98924065 1.47266853 ... 1.4846946  1.04623365 0.65402335]
(262656,)
------
5472
[2.51323962 1.98924065 1.47266853 ... 1.03011405 0.6214124  0.37574801]
(262704,)
------
5473
[2.51323962 1.98924065 1.47266853 ... 0.57756633 0.32516083 0.3361744 ]
(262752,)
------
5474
[2.51323962 1.98924065 1.47266853 ... 0.30096972 0.30289254 0.62931925]
(262800,)
------
5475
[2.51323962 1.98924065 1.47266853 ... 0.22232173 0.55382538 1.07723081]
(262848,)
------
5476
[2.51323962 1.98924065 1.47266853 ... 0.59529901 1.11629295 1.57202625]
(262896,)
------
5477
[2.51323962 1.98924065 1.47266853 ... 1.12252033 1.59187186 

[2.51323962 1.98924065 1.47266853 ... 1.71376383 2.13779163 2.38614941]
(266640,)
------
5555
[2.51323962 1.98924065 1.47266853 ... 2.13250613 2.38745904 2.38988662]
(266688,)
------
5556
[2.51323962 1.98924065 1.47266853 ... 2.38592172 2.3935163  2.05254388]
(266736,)
------
5557
[2.51323962 1.98924065 1.47266853 ... 2.39140129 2.04149938 1.55677783]
(266784,)
------
5558
[2.51323962 1.98924065 1.47266853 ... 2.05350709 1.56803358 1.1531738 ]
(266832,)
------
5559
[2.51323962 1.98924065 1.47266853 ... 1.57079065 1.14909744 0.72751552]
(266880,)
------
5560
[2.51323962 1.98924065 1.47266853 ... 1.14963067 0.72514099 0.42812851]
(266928,)
------
5561
[2.51323962 1.98924065 1.47266853 ... 0.74156684 0.4466947  0.37889719]
(266976,)
------
5562
[2.51323962 1.98924065 1.47266853 ... 0.46654591 0.39033386 0.66013372]
(267024,)
------
5563
[2.51323962 1.98924065 1.47266853 ... 0.4037526  0.67894047 1.17149401]
(267072,)
------
5564
[2.51323962 1.98924065 1.47266853 ... 0.67886752 1.17559886 

[2.51323962 1.98924065 1.47266853 ... 2.2168901  2.09375501 1.62790024]
(270912,)
------
5644
[2.51323962 1.98924065 1.47266853 ... 2.10663629 1.63724339 1.12238848]
(270960,)
------
5645
[2.51323962 1.98924065 1.47266853 ... 1.6426605  1.12245166 0.61437052]
(271008,)
------
5646
[2.51323962 1.98924065 1.47266853 ... 1.08390272 0.56583571 0.04525526]
(271056,)
------
5647
[ 2.51323962  1.98924065  1.47266853 ...  0.5409646   0.01162344
 -0.31016386]
(271104,)
------
5648
[ 2.51323962  1.98924065  1.47266853 ...  0.03106296 -0.29001114
 -0.24525519]
(271152,)
------
5649
[ 2.51323962  1.98924065  1.47266853 ... -0.28811586 -0.24334335
  0.228266  ]
(271200,)
------
5650
[ 2.51323962  1.98924065  1.47266853 ... -0.23424911  0.23048295
  0.90516329]
(271248,)
------
5651
[2.51323962 1.98924065 1.47266853 ... 0.23966418 0.92060351 1.49470961]
(271296,)
------
5652
[2.51323962 1.98924065 1.47266853 ... 0.91344833 1.48449993 1.99785519]
(271344,)
------
5653
[2.51323962 1.98924065 1.4726685

[2.51323962 1.98924065 1.47266853 ... 2.11840248 1.72731447 1.35171115]
(275136,)
------
5732
[2.51323962 1.98924065 1.47266853 ... 1.71215582 1.3423301  0.98906308]
(275184,)
------
5733
[2.51323962 1.98924065 1.47266853 ... 1.33917236 0.9833892  0.6607129 ]
(275232,)
------
5734
[2.51323962 1.98924065 1.47266853 ... 0.96821451 0.64018255 0.46226564]
(275280,)
------
5735
[2.51323962 1.98924065 1.47266853 ... 0.61867714 0.43994769 0.49786153]
(275328,)
------
5736
[2.51323962 1.98924065 1.47266853 ... 0.43494284 0.49848819 0.81117064]
(275376,)
------
5737
[2.51323962 1.98924065 1.47266853 ... 0.48006901 0.79191828 1.2089777 ]
(275424,)
------
5738
[2.51323962 1.98924065 1.47266853 ... 0.78161079 1.19436693 1.52962506]
(275472,)
------
5739
[2.51323962 1.98924065 1.47266853 ... 1.17854428 1.50856113 1.79190111]
(275520,)
------
5740
[2.51323962 1.98924065 1.47266853 ... 1.50542605 1.78333127 1.92161214]
(275568,)
------
5741
[2.51323962 1.98924065 1.47266853 ... 1.78140748 1.92538726 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19168, 48)

In [52]:
test_ds_predict_q50_supervised[0:3]

array([[2.51323962, 1.98924065, 1.47266853, 0.98042303, 0.51623917,
        0.31311059, 0.4915283 , 1.02235365, 1.60488594, 2.09089375,
        2.51266265, 2.67695332, 2.51126575, 1.99715519, 1.43865931,
        0.93535393, 0.43923306, 0.13391285, 0.17786956, 0.6562708 ,
        1.31081212, 1.86701143, 2.39162993, 2.7447567 , 2.78552365,
        2.42154932, 1.80927503, 1.30479717, 0.81478333, 0.39877805,
        0.26570162, 0.55420023, 1.15904009, 1.71582699, 2.21274495,
        2.56378078, 2.63389587, 2.33708477, 1.76076174, 1.26424015,
        0.74313277, 0.25665909, 0.05603945, 0.22033411, 0.79564714,
        1.44398463, 1.98395431, 2.48369431],
       [2.0324924 , 1.50960457, 1.02110279, 0.53950405, 0.32399324,
        0.4993751 , 1.03678083, 1.63141668, 2.12082982, 2.54747581,
        2.71403599, 2.55012417, 2.040766  , 1.47410297, 0.96508646,
        0.46053386, 0.14106727, 0.17570503, 0.6460464 , 1.30219412,
        1.86458623, 2.39635301, 2.76107407, 2.8113625 , 2.45195532,
   

#### actual y

In [53]:
from helper import series_to_supervised, stage_series_to_supervised

In [54]:
test_df_supervised = series_to_supervised(test_df[['WS_S1']], n_hours, K)

In [55]:
test_df_supervised.iloc[:, -K:].shape

(19168, 48)

In [56]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [57]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+38),var1(t+39),var1(t+40),var1(t+41),var1(t+42),var1(t+43),var1(t+44),var1(t+45),var1(t+46),var1(t+47)
77217,2.53,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,...,1.62,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48
77218,1.92,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,...,1.06,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80
77219,1.43,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,...,0.46,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70
77220,0.92,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,...,-0.07,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19
77221,0.50,0.20,0.47,0.98,1.62,2.12,2.53,2.75,2.59,2.05,...,-0.16,0.10,0.68,1.36,1.98,2.48,2.80,2.70,2.19,1.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,1.95,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,...,2.36,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24
96381,1.96,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,...,2.18,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62
96382,1.62,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,...,1.69,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98
96383,1.14,0.73,0.15,-0.20,-0.28,-0.05,0.52,1.13,1.59,1.99,...,1.14,0.64,0.11,0.04,0.18,0.77,1.24,1.62,1.98,2.13


#### performance on test set

In [58]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.157
RMSE = 0.209
